# DistilBERT

### Installs and imports

In [ ]:
!pip install datasets transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import matplotlib.pyplot as plt
import nltk
import numpy as np
import os
import pandas as pd
import seaborn as sns
import string
import time
import torch
import torch.optim as optim
import torch.nn.functional as F

from datasets import load_dataset
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report, ConfusionMatrixDisplay, matthews_corrcoef
from torch.nn import CosineEmbeddingLoss
from torch.utils.data import DataLoader
from transformers import (
    AutoModelForMaskedLM,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BertForSequenceClassification,
    BertTokenizer,
    DataCollatorForLanguageModeling,
    DataCollatorWithPadding,
    DistilBertConfig,
    TrainingArguments,
    Trainer
)
from tqdm import tqdm

save_path = "/content/drive/MyDrive/CS4782FinalProject/final_train"
save_model_folder_name = "final_distilled_student_model_wikitext"
model_path = os.path.join(save_path, save_model_folder_name)
checkpoint_path = os.path.join(save_path, "checkpoint.pth")

# Distillation using Masked Language Modelling

In [ ]:
class DistilBertConfig:
    teacher_model = "bert-base-uncased"
    student_model = "distilbert-base-uncased"

    device = "cuda" if torch.cuda.is_available() else "cpu"

    batch_size = 16
    learning_rate = 5e-5
    epochs = 10
    max_len = 128
    mlm_prob = 0.15

config = DistilBertConfig()

In [ ]:
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
tokenizer = AutoTokenizer.from_pretrained(config.teacher_model)

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=config.max_len)

tokenized_dataset = {
    split: dataset[split].map(tokenize_function, batched=True, remove_columns=["text"])
    for split in ["train", "validation", "test"]
}

for split in tokenized_dataset:
    tokenized_dataset[split].set_format(type="torch", columns=["input_ids", "attention_mask"])

# Data collator will mask tokens for us
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=config.mlm_prob)

train_loader = DataLoader(tokenized_dataset["train"], batch_size=config.batch_size, shuffle=True, collate_fn=data_collator)
val_loader = DataLoader(tokenized_dataset["validation"], batch_size=config.batch_size, shuffle=False, collate_fn=data_collator)
test_loader = DataLoader(tokenized_dataset["test"], batch_size=config.batch_size, shuffle=False, collate_fn=data_collator)

In [ ]:
teacher = AutoModelForMaskedLM.from_pretrained(config.teacher_model).to(config.device)
teacher.eval()

student = AutoModelForMaskedLM.from_pretrained(config.student_model).to(config.device)

In [ ]:
def copy_every_other_layer(student, teacher):
    """
    Copy every other teacher layer into the student.
    Assumes student has half as many encoder layers as teacher.
    """
    teacher_layers = teacher.bert.encoder.layer
    student_layers = student.distilbert.transformer.layer

    # Copy every other teacher layer into student
    for i, layer in enumerate(student_layers):
        teacher_layer = teacher_layers[i * 2]
        layer.load_state_dict(teacher_layer.state_dict(), strict=False)

copy_every_other_layer(student, teacher)

In [ ]:
def distill_loss(student_logits,
                 teacher_logits,
                 student_hidden,
                 teacher_hidden,
                 labels,
                 temperature=2.0,
                 alpha=0.5,
                 beta=0.5,
                 gamma=1.0):

    cosine_loss_fn = CosineEmbeddingLoss()

    # Only compute loss on masked tokens
    mask = labels != -100
    if mask.sum() == 0:
      print("mask sum is 0")
      return torch.tensor(0.0, device=student_logits.device, requires_grad=True)

    #### MLM Loss ####
    ce_loss = F.cross_entropy(student_logits.view(-1, student_logits.size(-1)), labels.view(-1), ignore_index=-100)

    # KL divergence between softened predictions
    student_probs = F.log_softmax(student_logits / temperature, dim=-1)
    teacher_probs = F.softmax(teacher_logits / temperature, dim=-1)

    kl_loss = F.kl_div(
        student_probs,
        teacher_probs,
        reduction="batchmean"
    ) * (temperature ** 2)

    # Cosine embedding loss on hidden states
    # Cosine embedding loss between intermediate hidden states
    losses = []
    teacher_layers = teacher_hidden[1::2]  # take every 2nd layer starting from index 1
    student_layers = student_hidden[1:]    # skip the student input embedding layer

    for t, s in zip(teacher_layers, student_layers):
        # Flatten hidden state in vectors of HIDDEN_STATE_SIZE for every token
        # in the batch (B, SEQ_LEN, HIDDEN_STATE_SIZE) -> (B x SEQ_LEN, HIDDEN_STATE_SIZE)
        t_flat = t.reshape(-1, t.size(-1))
        s_flat = s.reshape(-1, s.size(-1))

        # Create a tensor of ones (B x SEQ_LEN) as the target cosine similarity
        # between teacher and student hidden states
        target = torch.ones(t_flat.size(0)).to(t.device)
        losses.append(cosine_loss_fn(s_flat, t_flat, target))

    cos_loss = torch.stack(losses).mean()

    total_loss = alpha * ce_loss + beta * kl_loss + gamma * cos_loss
    return total_loss, {
        "ce_loss": ce_loss.item(),
        "kd_loss": kl_loss.item(),
        "cos_loss": cos_loss.item()
    }

In [ ]:
optimizer = optim.AdamW(student.parameters(), lr=config.learning_rate)

In [ ]:
# Save a training checkpoint (for resuming training later)
def save_checkpoint(model, optimizer, epoch, path):
    if not os.path.exists(os.path.dirname(path)):
        os.makedirs(os.path.dirname(path))

    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, path)
    print(f"Checkpoint saved to {path}")

start_epoch = 0
if os.path.exists(checkpoint_path):
    print(f"Loading checkpoint from {checkpoint_path}")
    checkpoint = torch.load(checkpoint_path, map_location=config.device)
    student.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    start_epoch = checkpoint["epoch"] + 1
    print(f"Resumed from epoch {start_epoch}")
else:
    print("Starting training from scratch")

In [ ]:
train_losses = []
eval_accuracies = []
epochs_times = []
start_time = time.time()

try:
    for epoch in range(start_epoch, config.epochs):
        student.train()
        total_loss = 0

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            input_ids = batch["input_ids"].to(config.device)
            attention_mask = batch["attention_mask"].to(config.device)
            labels = batch["labels"].to(config.device)

            # Teacher outputs
            with torch.no_grad():
                teacher_outputs = teacher(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    output_hidden_states=True
                )
                teacher_logits = teacher_outputs.logits
                teacher_hidden_states = teacher_outputs.hidden_states

            # Student outputs
            student_outputs = student(
                input_ids=input_ids,
                attention_mask=attention_mask,
                output_hidden_states=True
            )
            student_logits = student_outputs.logits
            student_hidden_states = student_outputs.hidden_states

             # Compute distillation loss (returns total_loss, loss_dict)
            loss, loss_dict = distill_loss(
                student_logits, teacher_logits,
                student_hidden_states, teacher_hidden_states,
                labels, temperature=2.0, alpha=0.5, beta=0.5, gamma=1.0
            )

            if torch.isnan(loss) or torch.isinf(loss):
                print("Skipping batch due to NaN/Inf loss")
                continue

            # Update model
            loss.backward()
            torch.nn.utils.clip_grad_norm_(student.parameters(), max_norm=1.0)
            optimizer.step()
            optimizer.zero_grad()

            total_loss += loss.item()

        # Track training loss
        avg_train_loss = total_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # Validation evaluation
        student.eval()
        correct = 0
        total_masked = 0

        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch["input_ids"].to(config.device)
                attention_mask = batch["attention_mask"].to(config.device)
                labels = batch["labels"].to(config.device)

                outputs = student(input_ids=input_ids, attention_mask=attention_mask)
                logits = outputs.logits
                predictions = logits.argmax(dim=-1)

                mask = labels != -100
                correct += (predictions[mask] == labels[mask]).sum().item()
                total_masked += mask.sum().item()

        mlm_accuracy = correct / total_masked if total_masked > 0 else 0.0
        eval_accuracies.append(mlm_accuracy)

        # Track timer
        epoch_time = time.time() - start_time
        epochs_times.append(epoch_time)

        # Print metrics
        print(f"Epoch {epoch+1}: Loss={avg_train_loss:.4f}, Acc={mlm_accuracy:.4f}, Time={epoch_time:.2f}s")

        # Save checkpoint after every epoch
        save_checkpoint(student, optimizer, epoch, checkpoint_path)

        # Reset timer for next epoch
        start_time = time.time()

except Exception as e:
    print(f"Exception occurred during training: {e}")
    print("Saving current model and checkpoint...")

    student.save_pretrained(model_path)
    tokenizer.save_pretrained(model_path)
    save_checkpoint(student, optimizer, epoch, checkpoint_path)

    print(f"Hugging Face model saved to {model_path}")

In [ ]:
student.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)
print(f"Hugging Face model saved to {model_path}")

save_checkpoint(student, optimizer, epoch=epoch, path=checkpoint_path)

In [ ]:
# Loss curve
loss_path = os.path.join(save_path, "loss_curve.png")

plt.figure()
plt.plot(range(1, config.epochs+1), train_losses, marker='o')
plt.title("Training Loss per Epoch")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid()
plt.savefig(loss_path)
plt.show()

# Accuracy curve
accuracy_path = os.path.join(save_path, "accuracy_curve.png")

plt.figure()
plt.plot(range(1, config.epochs+1), eval_accuracies, marker='o', color='green')
plt.title("Validation Accuracy per Epoch")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.grid()
plt.savefig(accuracy_path)
plt.show()

In [ ]:
print(train_losses)
print(eval_accuracies)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

teacher_params = count_parameters(teacher)
student_params = count_parameters(student)
compression_ratio = teacher_params / student_params

print(f"Teacher Params: {teacher_params:,}")
print(f"Student Params: {student_params:,}")
print(f"Compression Ratio: {compression_ratio:.2f}x")

# Finetuning and Evaluation

In [ ]:
class FineTuningConfig:
    device = "cuda" if torch.cuda.is_available() else "cpu"
    batch_size = 16
    epochs = 3
    learning_rate = 2e-5

config = FineTuningConfig()

In [ ]:
def evaluate_model(model, dataloader):
    model.eval()
    preds = []
    labels = []
    times = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(config.device)
            attention_mask = batch["attention_mask"].to(config.device)
            batch_labels = batch["labels"].to(config.device)

            start_time = time.time()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            end_time = time.time()
            total_time = end_time - start_time
            times.append(total_time)

            logits = outputs.logits
            preds.append(logits.argmax(dim=-1).cpu())
            labels.append(batch_labels.cpu())

    avg_inference_time = sum(times) / len(times)

    preds = torch.cat(preds)
    labels = torch.cat(labels)

    acc = accuracy_score(labels, preds)
    mcc = matthews_corrcoef(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    conf_matrix = confusion_matrix(labels, preds)
    report = classification_report(labels, preds, digits=4)

    f1_acc_avg = (f1 + acc) / 2

    return {
        "accuracy": acc,
        "matthews_corrcoef": mcc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "f1_acc_avg": f1_acc_avg,
        "confusion_matrix": conf_matrix,
        "report": report,
        "avg_inference_time": avg_inference_time,
    }

## Finetune for IMDb sequence classification

In [ ]:
imdb_path = os.path.join(save_path, "imdb_finetune_final")

In [ ]:
imdb = load_dataset("imdb")

tokenizer = AutoTokenizer.from_pretrained(model_path,local_files_only=True)

def preprocess(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=256)

encoded_imdb = imdb.map(preprocess, batched=True)
encoded_imdb.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

data_collator = DataCollatorWithPadding(tokenizer)

train_loader = DataLoader(encoded_imdb["train"], shuffle=True, batch_size=config.batch_size, collate_fn=data_collator)
test_loader = DataLoader(encoded_imdb["test"], batch_size=config.batch_size, collate_fn=data_collator)

In [ ]:
student_model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2).to(config.device)

training_args = TrainingArguments(
    output_dir=os.path.join(imdb_path,"results-student"),
    eval_strategy="epoch",
    learning_rate=config.learning_rate,
    per_device_train_batch_size=config.batch_size,
    per_device_eval_batch_size=config.batch_size,
    num_train_epochs=config.epochs,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir=os.path.join(imdb_path,"logs-student"),
    logging_steps=100,
    report_to="none"
)

trainer = Trainer(
    model=student_model,
    args=training_args,
    train_dataset=encoded_imdb["train"],
    eval_dataset=encoded_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

student_imdb_path = os.path.join(imdb_path,"student_model")
student_model.save_pretrained(student_imdb_path)

In [ ]:
teacher_model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).to(config.device)

training_args = TrainingArguments(
    output_dir=os.path.join(imdb_path,"results-teacher"),
    eval_strategy="epoch",
    learning_rate=config.learning_rate,
    per_device_train_batch_size=config.batch_size,
    per_device_eval_batch_size=config.batch_size,
    num_train_epochs=config.epochs,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir=os.path.join(imdb_path,"logs-teacher"),
    logging_steps=100,
    report_to="none"
)

trainer = Trainer(
    model=teacher_model,
    args=training_args,
    train_dataset=encoded_imdb["train"],
    eval_dataset=encoded_imdb["test"],
    tokenizer=tokenizer, # student tokenizer
    data_collator=data_collator,
)

trainer.train()

teacher_imdb_path = os.path.join(imdb_path,"teacher_model")
teacher_model.save_pretrained(teacher_imdb_path)

## IMDb Evaluation

In [ ]:
imdb_eval_path = os.path.join(imdb_path, "eval")

student_imdb_path = os.path.join(imdb_path,"student_model")
teacher_imdb_path = os.path.join(imdb_path,"teacher_model")

student = AutoModelForSequenceClassification.from_pretrained(student_imdb_path).to(config.device)
teacher = AutoModelForSequenceClassification.from_pretrained(teacher_imdb_path).to(config.device)
tokenizer = AutoTokenizer.from_pretrained(model_path)

student_imdb_results = evaluate_model(student, test_loader)
teacher_imdb_results = evaluate_model(teacher, test_loader)

In [ ]:
################################### Accuracy ###################################

student_imdb_acc = student_imdb_results["accuracy"]
teacher_imdb_acc = teacher_imdb_results["accuracy"]

imdb_acc_retained_pct = 100 * student_imdb_acc / teacher_imdb_acc

print(f"Teacher Accuracy: {teacher_imdb_acc:.4f}")
print(f"Student Accuracy: {student_imdb_acc:.4f}")
print(f"Student retained {imdb_acc_retained_pct:.2f}% of teacher's performance.")

################################################################################

In [ ]:
########################## Speed Up and Inference Time #########################

student_imdb_inference_time = student_imdb_results["avg_inference_time"]
teacher_imdb_inference_time = teacher_imdb_results["avg_inference_time"]

print(f"Student Avg Inference Time: {student_imdb_inference_time*1000:.2f} ms")
print(f"Teacher Avg Inference Time: {teacher_imdb_inference_time*1000:.2f} ms")

imdb_speedup = teacher_imdb_inference_time / student_imdb_inference_time
print(f"Speedup: {imdb_speedup:.2f}x faster")
################################################################################

In [ ]:
###################### Student Confusion Matrix and Report #####################

student_imdb_confusion_matrix = student_imdb_results["confusion_matrix"]
student_imdb_report = student_imdb_results["report"]
imdb_cm_matrix_path = os.path.join(imdb_eval_path, "imdb_confusion_matrix.png")
os.makedirs(os.path.dirname(imdb_cm_matrix_path), exist_ok=True)

disp = ConfusionMatrixDisplay(confusion_matrix=student_imdb_confusion_matrix)
disp.plot(cmap="Blues")
plt.title("Student Model Confusion Matrix")
plt.savefig(imdb_cm_matrix_path)
plt.show()

print("Classification Report:\n")
print(student_imdb_report)

classification_report_path = os.path.join(imdb_eval_path, "student_classification_report_imdb.txt")
with open(classification_report_path, "w") as f:
    f.write(student_imdb_report)

################################################################################

################################## Comparsion ##################################
imdb_comparison_df = pd.DataFrame([
    {
        "Model": "Student",
        **student_imdb_results,
    },
    {
        "Model": "Teacher",
        **teacher_imdb_results,
    }
])

imdb_comparison_df = imdb_comparison_df[[
    "Model",
    "accuracy",
    "precision",
    "recall",
    "f1",
    "avg_inference_time"
]]

print(imdb_comparison_df.to_markdown(index=False))

imdb_comparsion_df_save_path = os.path.join(imdb_eval_path, "imdb_comparison.csv")
imdb_comparison_df.to_csv(imdb_comparsion_df_save_path, index=False)
print(f"Saved CSV to {imdb_comparsion_df_save_path}")

################################################################################

In [ ]:
""""
OpenAI. (2025). ChatGPT (May 2025 version). Code generated for token analysis 
and attention heatmap visualization in BERT and DistilBERT models. Retrieved 
May 3, 2025 from ChatGPT.com
"""

nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

def get_top_k_token_indices(attn_tensor, tokens, top_k=16):
    """Return indices of top-k most attended-to meaningful tokens."""
    avg_attn = attn_tensor.mean(dim=0).cpu().numpy()
    incoming_attention = avg_attn.sum(axis=0)

    def is_valid_word(tok):
        return (
            tok.isalpha() and
            tok.lower() not in stop_words and
            not tok.startswith("##") and
            not tok.startswith("[")
        )

    valid_tokens = [(i, tok) for i, tok in enumerate(tokens) if is_valid_word(tok)]
    sorted_valid = sorted(valid_tokens, key=lambda x: incoming_attention[x[0]], reverse=True)
    return [i for i, _ in sorted_valid[:top_k]]

def plot_attention_heatmap_top_k_only(attn_tensor, tokens, title, ax, top_indices, highlight_index=None):
    """Plots a k x k attention heatmap based on provided top token indices."""
    avg_attn = attn_tensor.mean(dim=0).cpu().numpy()
    cropped_attn = avg_attn[np.ix_(top_indices, top_indices)]
    cropped_tokens = [tokens[i] for i in top_indices]

    sns.heatmap(cropped_attn, xticklabels=cropped_tokens, yticklabels=cropped_tokens,
                ax=ax, cmap="Blues", cbar=False)
    ax.set_title(title, fontsize=10)
    ax.tick_params(labelsize=6)
    ax.tick_params(axis='x', rotation=90)

    if highlight_index is not None and highlight_index in top_indices:
        idx = top_indices.index(highlight_index)
        ax.get_xticklabels()[idx].set_color("red")
        ax.get_xticklabels()[idx].set_fontweight("bold")
        ax.get_yticklabels()[idx].set_color("red")
        ax.get_yticklabels()[idx].set_fontweight("bold")

mistake_examples = []
teacher.eval()
student.eval()

imdb_subset = imdb["test"].select(range(1000))

for example in imdb_subset:
    text = example["text"]
    label = example["label"]

    inputs_raw = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)
    inputs = {k: v.to(config.device) for k, v in inputs_raw.items()}

    if "distilbert" in student.base_model_prefix.lower() and "token_type_ids" in inputs:
        del inputs["token_type_ids"]

    with torch.no_grad():
        t_logits = teacher(**inputs).logits
        s_logits = student(**inputs).logits

    t_pred = t_logits.argmax(dim=-1).item()
    s_pred = s_logits.argmax(dim=-1).item()

    if t_pred == label and s_pred != label:
        mistake_examples.append({
            "text": text,
            "label": label,
            "teacher_pred": t_pred,
            "student_pred": s_pred,
            "inputs": inputs
        })

    if len(mistake_examples) >= 25:
        break

print(f"Found {len(mistake_examples)} teacher-correct, student-wrong examples.")

for i, ex in enumerate(mistake_examples):
    inputs = ex["inputs"]
    input_ids = inputs["input_ids"]
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

    with torch.no_grad():
        teacher_output = teacher(**inputs, output_attentions=True)
        student_output = student(**inputs, output_attentions=True)

    teacher_attn = teacher_output.attentions[-1][0]
    student_attn = student_output.attentions[-1][0]

    teacher_top = get_top_k_token_indices(teacher_attn, tokens, top_k=100)
    student_top = get_top_k_token_indices(student_attn, tokens, top_k=100)
    top_union = sorted(set(teacher_top + student_top))[:16]

    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    plot_attention_heatmap_top_k_only(teacher_attn, tokens, "Teacher Attention", axes[0], top_union, highlight_index=0)
    plot_attention_heatmap_top_k_only(student_attn, tokens, "Student Attention", axes[1], top_union, highlight_index=0)

    fig.suptitle(f"Example {i+1}: Label={ex['label']} | Teacher={ex['teacher_pred']} | Student={ex['student_pred']}", fontsize=12)
    fig.tight_layout(rect=[0, 0, 1, 0.95])

    save_path = os.path.join(imdb_eval_path, f"attention_example_topk_{i+1}.png")
    fig.savefig(save_path, dpi=300)
    plt.close(fig)

    print(f"Saved: {save_path}")

## GLuE Evaluation

### Finetune for CoLA Dataset


In [ ]:
cola_path = os.path.join(save_path, "cola_finetune_final")

In [ ]:
cola = load_dataset("glue", "cola")

tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)

def preprocess(example):
    return tokenizer(example["sentence"], truncation=True, padding="max_length", max_length=256)

encoded_cola = cola.map(preprocess, batched=True)
encoded_cola.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

data_collator = DataCollatorWithPadding(tokenizer)

train_loader = DataLoader(encoded_cola["train"], shuffle=True, batch_size=config.batch_size, collate_fn=data_collator)
test_loader = DataLoader(encoded_cola["validation"], batch_size=config.batch_size, collate_fn=data_collator)

In [ ]:
student_model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2).to(config.device)

training_args = TrainingArguments(
    output_dir=os.path.join(cola_path, "results-student"),
    eval_strategy="epoch",
    learning_rate=config.learning_rate,
    per_device_train_batch_size=config.batch_size,
    per_device_eval_batch_size=config.batch_size,
    num_train_epochs=config.epochs,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir=os.path.join(cola_path, "logs-student"),
    logging_steps=100,
    report_to="none"
)

trainer = Trainer(
    model=student_model,
    args=training_args,
    train_dataset=encoded_cola["train"],
    eval_dataset=encoded_cola["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

student_cola_path = os.path.join(cola_path, "student_model")
student_model.save_pretrained(student_cola_path)

In [ ]:
teacher_model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).to(config.device)

training_args = TrainingArguments(
    output_dir=os.path.join(cola_path, "results-teacher"),
    eval_strategy="epoch",
    learning_rate=config.learning_rate,
    per_device_train_batch_size=config.batch_size,
    per_device_eval_batch_size=config.batch_size,
    num_train_epochs=config.epochs,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir=os.path.join(cola_path, "logs-teacher"),
    logging_steps=100,
    report_to="none"
)

trainer = Trainer(
    model=teacher_model,
    args=training_args,
    train_dataset=encoded_cola["train"],
    eval_dataset=encoded_cola["validation"],
    tokenizer=tokenizer,  # student tokenizer
    data_collator=data_collator,
)

trainer.train()

teacher_cola_path = os.path.join(cola_path, "teacher_model")
teacher_model.save_pretrained(teacher_cola_path)

### CoLA Evaluation

In [ ]:
cola_eval_path = os.path.join(cola_path, "eval")

student_cola_path = os.path.join(cola_path,"student_model")
teacher_cola_path = os.path.join(cola_path,"teacher_model")

student = AutoModelForSequenceClassification.from_pretrained(student_cola_path).to(config.device)
teacher = AutoModelForSequenceClassification.from_pretrained(teacher_cola_path).to(config.device)
tokenizer = AutoTokenizer.from_pretrained(model_path)

student_cola_results = evaluate_model(student, test_loader)
teacher_cola_results = evaluate_model(teacher, test_loader)

In [ ]:
################################### Accuracy ###################################

student_cola_acc = student_cola_results["accuracy"]
teacher_cola_acc = teacher_cola_results["accuracy"]

cola_acc_retained_pct = 100 * student_cola_acc / teacher_cola_acc

print(f"Teacher Accuracy: {teacher_cola_acc:.4f}")
print(f"Student Accuracy: {student_cola_acc:.4f}")
print(f"Student retained {cola_acc_retained_pct:.2f}% of teacher's performance.")

################################################################################

In [ ]:
########################## Speed Up and Inference Time #########################

student_cola_inference_time = student_cola_results["avg_inference_time"]
teacher_cola_inference_time = teacher_cola_results["avg_inference_time"]

print(f"Student Avg Inference Time: {student_cola_inference_time*1000:.2f} ms")
print(f"Teacher Avg Inference Time: {teacher_cola_inference_time*1000:.2f} ms")

cola_speedup = teacher_cola_inference_time / student_cola_inference_time
print(f"Speedup: {cola_speedup:.2f}x faster")
################################################################################

In [ ]:
################################# Matthews Coefficient ###############################

student_cola_mcc = student_cola_results["matthews_corrcoef"]
teacher_cola_mcc = teacher_cola_results["matthews_corrcoef"]

cola_mcc_retained_pct = 100 * student_cola_mcc / teacher_cola_mcc if teacher_cola_mcc != 0 else 0

print(f"Teacher MCC: {teacher_cola_mcc:.4f}")
print(f"Student MCC: {student_cola_mcc:.4f}")
print(f"Student retained {cola_mcc_retained_pct:.2f}% of teacher's MCC performance.")
################################################################################

In [ ]:
###################### Student Confusion Matrix and Report #####################

student_cola_confusion_matrix = student_cola_results["confusion_matrix"]
student_cola_report = student_cola_results["report"]
cola_cm_matrix_path = os.path.join(cola_eval_path, "cola_confusion_matrix.png")
os.makedirs(os.path.dirname(cola_cm_matrix_path), exist_ok=True)

disp = ConfusionMatrixDisplay(confusion_matrix=student_cola_confusion_matrix)
disp.plot(cmap="Blues")
plt.title("Student Model Confusion Matrix")
plt.savefig(cola_cm_matrix_path)
plt.show()

print("Classification Report:\n")
print(student_cola_report)

classification_report_path = os.path.join(cola_eval_path, "student_classification_report_cola.txt")
with open(classification_report_path, "w") as f:
    f.write(student_cola_report)

################################################################################

################################## Comparsion ##################################
cola_comparison_df = pd.DataFrame([
    {
        "Model": "Student",
        **student_cola_results,
    },
    {
        "Model": "Teacher",
        **teacher_cola_results,
    }
])

cola_comparison_df = cola_comparison_df[[
    "Model",
    "matthews_corrcoef",
    "accuracy",
    "precision",
    "recall",
    "f1",
    "avg_inference_time"
]]

print(cola_comparison_df.to_markdown(index=False))

cola_comparsion_df_save_path = os.path.join(cola_eval_path, "cola_comparison.csv")
cola_comparison_df.to_csv(cola_comparsion_df_save_path, index=False)
print(f"Saved CSV to {cola_comparsion_df_save_path}")

################################################################################

### Finetune for MRPC Dataset

In [ ]:
mrpc_path = os.path.join(save_path, "mrpc_finetune_final")

In [ ]:
mrpc = load_dataset("glue", "mrpc")

tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)

def preprocess(example):
    return tokenizer(
        example["sentence1"],
        example["sentence2"],
        truncation=True,
        padding="max_length",
        max_length=256
    )

encoded_mrpc = mrpc.map(preprocess, batched=True)
encoded_mrpc.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

data_collator = DataCollatorWithPadding(tokenizer)

train_loader = DataLoader(encoded_mrpc["train"], shuffle=True, batch_size=config.batch_size, collate_fn=data_collator)
test_loader = DataLoader(encoded_mrpc["validation"], batch_size=config.batch_size, collate_fn=data_collator)

In [ ]:
student_model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2).to(config.device)

training_args = TrainingArguments(
    output_dir=os.path.join(mrpc_path, "results-student"),
    eval_strategy="epoch",
    learning_rate=config.learning_rate,
    per_device_train_batch_size=config.batch_size,
    per_device_eval_batch_size=config.batch_size,
    num_train_epochs=config.epochs,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir=os.path.join(mrpc_path, "logs-student"),
    logging_steps=100,
    report_to="none"
)

trainer = Trainer(
    model=student_model,
    args=training_args,
    train_dataset=encoded_mrpc["train"],
    eval_dataset=encoded_mrpc["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

student_mrpc_path = os.path.join(mrpc_path, "student_model")
student_model.save_pretrained(student_mrpc_path)

In [ ]:
teacher_model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).to(config.device)

training_args = TrainingArguments(
    output_dir=os.path.join(mrpc_path, "results-teacher"),
    eval_strategy="epoch",
    learning_rate=config.learning_rate,
    per_device_train_batch_size=config.batch_size,
    per_device_eval_batch_size=config.batch_size,
    num_train_epochs=config.epochs,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir=os.path.join(mrpc_path, "logs-teacher"),
    logging_steps=100,
    report_to="none"
)

trainer = Trainer(
    model=teacher_model,
    args=training_args,
    train_dataset=encoded_mrpc["train"],
    eval_dataset=encoded_mrpc["validation"],
    tokenizer=tokenizer,  # student tokenizer
    data_collator=data_collator,
)

trainer.train()

teacher_mrpc_path = os.path.join(mrpc_path, "teacher_model")
teacher_model.save_pretrained(teacher_mrpc_path)

### MRPC Evaluation

In [ ]:
mrpc_eval_path = os.path.join(mrpc_path, "eval")

student_mrpc_path = os.path.join(mrpc_path,"student_model")
teacher_mrpc_path = os.path.join(mrpc_path,"teacher_model")

student = AutoModelForSequenceClassification.from_pretrained(student_mrpc_path).to(config.device)
teacher = AutoModelForSequenceClassification.from_pretrained(teacher_mrpc_path).to(config.device)
tokenizer = AutoTokenizer.from_pretrained(model_path)

student_mrpc_results = evaluate_model(student, test_loader)
teacher_mrpc_results = evaluate_model(teacher, test_loader)

In [ ]:
################################### Accuracy ###################################

student_mrpc_acc = student_mrpc_results["accuracy"]
teacher_mrpc_acc = teacher_mrpc_results["accuracy"]

mrpc_acc_retained_pct = 100 * student_mrpc_acc / teacher_mrpc_acc

print(f"Teacher Accuracy: {teacher_mrpc_acc:.4f}")
print(f"Student Accuracy: {student_mrpc_acc:.4f}")
print(f"Student retained {mrpc_acc_retained_pct:.2f}% of teacher's performance.")

################################################################################

In [ ]:
########################## Speed Up and Inference Time #########################

student_mrpc_inference_time = student_mrpc_results["avg_inference_time"]
teacher_mrpc_inference_time = teacher_mrpc_results["avg_inference_time"]

print(f"Student Avg Inference Time: {student_mrpc_inference_time*1000:.2f} ms")
print(f"Teacher Avg Inference Time: {teacher_mrpc_inference_time*1000:.2f} ms")

mrpc_speedup = teacher_mrpc_inference_time / student_mrpc_inference_time
print(f"Speedup: {mrpc_speedup:.2f}x faster")
################################################################################

In [ ]:
############################### F1-Accuracy Average ############################

student_mrpc_f1_acc_avg = student_mrpc_results["f1_acc_avg"]
teacher_mrpc_f1_acc_avg = teacher_mrpc_results["f1_acc_avg"]

mrpc_f1_acc_retained_pct = 100 * student_mrpc_f1_acc_avg / teacher_mrpc_f1_acc_avg if teacher_mrpc_f1_acc_avg != 0 else 0

print(f"Teacher F1/Accuracy: {teacher_mrpc_f1_acc_avg:.4f}")
print(f"Student F1/Accuracy: {student_mrpc_f1_acc_avg:.4f}")
print(f"Student retained {mrpc_f1_acc_retained_pct:.2f}% of teacher's F1/Accuracy performance.")
################################################################################

In [ ]:
###################### Student Confusion Matrix and Report #####################

student_mrpc_confusion_matrix = student_mrpc_results["confusion_matrix"]
student_mrpc_report = student_mrpc_results["report"]
mrpc_cm_matrix_path = os.path.join(mrpc_eval_path, "mrpc_confusion_matrix.png")
os.makedirs(os.path.dirname(mrpc_cm_matrix_path), exist_ok=True)

disp = ConfusionMatrixDisplay(confusion_matrix=student_mrpc_confusion_matrix)
disp.plot(cmap="Blues")
plt.title("Student Model Confusion Matrix")
plt.savefig(mrpc_cm_matrix_path)
plt.show()

print("Classification Report:\n")
print(student_mrpc_report)

classification_report_path = os.path.join(mrpc_eval_path, "student_classification_report_mrpc.txt")
with open(classification_report_path, "w") as f:
    f.write(student_mrpc_report)

################################################################################

################################## Comparsion ##################################
mrpc_comparison_df = pd.DataFrame([
    {
        "Model": "Student",
        **student_mrpc_results,
    },
    {
        "Model": "Teacher",
        **teacher_mrpc_results,
    }
])

mrpc_comparison_df = mrpc_comparison_df[[
    "Model",
    "accuracy",
    "precision",
    "recall",
    "f1",
    "f1_acc_avg",
    "avg_inference_time"
]]

print(mrpc_comparison_df.to_markdown(index=False))

mrpc_comparsion_df_save_path = os.path.join(mrpc_eval_path, "mrpc_comparison.csv")
mrpc_comparison_df.to_csv(mrpc_comparsion_df_save_path, index=False)
print(f"Saved CSV to {mrpc_comparsion_df_save_path}")

################################################################################

# Overall Model Evaluation and Paper Comparsion

In [ ]:
distilbert_imdb_paper_results = {
    "Model": "DistilBERT-IMDB",
    "score": 92.82,
}

bert_imdb_paper_results = {
    "Model": "BERT-IMDB",
    "score": 93.46,
}

distilbert_cola_paper_results = {
    "Model": "DistilBERT-CoLA",
    "score": 51.3,
}

bert_cola_paper_results = {
    "Model": "BERT-CoLA",
    "score": 56.3,
}

distilbert_mrpc_paper_results = {
    "Model": "DistilBERT-MRPC",
    "score": 87.5,
}

bert_mrpc_paper_results = {
    "Model": "BERT-MRPC",
    "score": 88.6,
}

distilbert_imdb_our_results = {
    "Model": "Our DistilBERT-IMDB",
    "score": student_imdb_acc*100,
}

bert_imdb_our_results = {
    "Model": "Our BERT-IMDB",
    "score": teacher_imdb_acc*100,
}

distilbert_cola_our_results = {
    "Model": "Our DistilBERT-CoLA",
    "score": student_cola_mcc*100,
}

bert_cola_our_results = {
    "Model": "Our BERT-CoLA",
    "score": teacher_cola_mcc*100,
}

distilbert_mrpc_our_results = {
    "Model": "Our DistilBERT-MRPC",
    "score": student_mrpc_f1_acc_avg*100,
}

bert_mrpc_our_results = {
    "Model": "Our BERT-MRPC",
    "score": teacher_mrpc_f1_acc_avg*100,
}

In [ ]:
def plot_comparisons(imdb_results, cola_results, mrpc_results):
    all_results = []

    for task_name, results in [
        ("IMDb", imdb_results),
        ("CoLA", cola_results),
        ("MRPC", mrpc_results),
    ]:
        for r in results:
            all_results.append({
                "Task": task_name,
                "Model": r["Model"],
                "Score": r["score"]
            })

    df = pd.DataFrame(all_results)

    base_models = {
        "Our DistilBERT": "#1f77b4",
        "Our BERT": "#ff7f0e",
        "DistilBERT": "#2ca02c",
        "BERT": "#9467bd",
    }

    def clean_model_label(model_str):
        if "Our DistilBERT" in model_str:
            return "Our DistilBERT"
        elif "Our BERT" in model_str:
            return "Our BERT"
        elif "DistilBERT" in model_str:
            return "DistilBERT"
        elif "BERT" in model_str:
            return "BERT"
        else:
            return model_str

    df["BaseModel"] = df["Model"].apply(clean_model_label)

    tasks = df["Task"].unique()
    models = df["BaseModel"].unique()
    n_models = len(models)
    x = np.arange(len(tasks))
    bar_width = 0.15

    fig, ax = plt.subplots(figsize=(10, 6))

    for i, model in enumerate(models):
        model_data = df[df["BaseModel"] == model]
        scores = [model_data[model_data["Task"] == task]["Score"].values[0] for task in tasks]
        offsets = x + (i - n_models / 2) * bar_width + bar_width / 2

        ax.bar(offsets, scores, width=bar_width, label=model, color=base_models[model])

    ax.set_xticks(x)
    ax.set_xticklabels(tasks)
    ax.set_ylabel("Score")
    ax.set_title("Model Performance Comparison Across Tasks")
    ax.grid(axis="y", linestyle="--", alpha=0.6)
    ax.legend(title="Model", loc="upper right")
    plt.tight_layout()

    plot_path = os.path.join(save_path,"performance_comparison_final.png")
    plt.savefig(plot_path, dpi=300, bbox_inches="tight")
    plt.show()

plot_comparisons(
    imdb_results=[
        distilbert_imdb_our_results,
        bert_imdb_our_results,
        distilbert_imdb_paper_results,
        bert_imdb_paper_results
    ],
    cola_results=[
        distilbert_cola_our_results,
        bert_cola_our_results,
        distilbert_cola_paper_results,
        bert_cola_paper_results
    ],
    mrpc_results=[
        distilbert_mrpc_our_results,
        bert_mrpc_our_results,
        distilbert_mrpc_paper_results,
        bert_mrpc_paper_results
    ]
)

In [ ]:
################################### Benchmark ##################################
benchmark_retained_pct = (cola_mcc_retained_pct + mrpc_f1_acc_retained_pct + imdb_acc_retained_pct )/3

print(f"Student retained {benchmark_retained_pct:.2f}% of teacher's performance.")

################################################################################

In [ ]:
########################## Speed Up and Inference Time #########################
avg_speedup = (cola_speedup + mrpc_speedup + imdb_speedup)/3

print(f"Speedup: {avg_speedup:.2f}x faster")
################################################################################